In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

In [4]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'w': 0, 'p': 1, 'y': 2, 's': 3, 'd': 4, 'o': 5, 'u': 6, 'm': 7, 'a': 8, 't': 9, ' ': 10, 'c': 11, 'n': 12, 'h': 13, 'b': 14, 'r': 15, 'k': 16, '.': 17, 'i': 18, ',': 19, 'l': 20, 'f': 21, "'": 22, 'g': 23, 'e': 24}


In [5]:
dic_size = len(char_dic)
print(f"문자 집합 크기 : {dic_size}")

문자 집합 크기 : 25


In [6]:
hidden_size = dic_size
sequence_length = 10
lr = 1e-1

In [7]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    print(f"{x_str} -> {y_str}")

    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

if you wan -> f you want
f you want ->  you want 
 you want  -> you want t
you want t -> ou want to
ou want to -> u want to 
u want to  ->  want to b
 want to b -> want to bu
want to bu -> ant to bui
ant to bui -> nt to buil
nt to buil -> t to build
t to build ->  to build 
 to build  -> to build a
to build a -> o build a 
o build a  ->  build a s
 build a s -> build a sh
build a sh -> uild a shi
uild a shi -> ild a ship
ild a ship -> ld a ship,
ld a ship, -> d a ship, 
d a ship,  ->  a ship, d
 a ship, d -> a ship, do
a ship, do ->  ship, don
 ship, don -> ship, don'
ship, don' -> hip, don't
hip, don't -> ip, don't 
ip, don't  -> p, don't d
p, don't d -> , don't dr
, don't dr ->  don't dru
 don't dru -> don't drum
don't drum -> on't drum 
on't drum  -> n't drum u
n't drum u -> 't drum up
't drum up -> t drum up 
t drum up  ->  drum up p
 drum up p -> drum up pe
drum up pe -> rum up peo
rum up peo -> um up peop
um up peop -> m up peopl
m up peopl ->  up people
 up people -> up people 


In [8]:
print(x_data[0])
print(y_data[0])

[18, 21, 10, 2, 5, 6, 10, 0, 8, 12]
[21, 10, 2, 5, 6, 10, 0, 8, 12, 9]


In [9]:
x_one_hot = torch.FloatTensor([np.eye(dic_size)[x] for x in x_data])
X = x_one_hot
Y = torch.LongTensor(y_data)

C:\Users\kck\AppData\Local\Temp\ipykernel_32276\3125409070.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  x_one_hot = torch.FloatTensor([np.eye(dic_size)[x] for x in x_data])


In [10]:
print(X.shape)
print(Y.shape)

torch.Size([170, 10, 25])
torch.Size([170, 10])


In [11]:
print(f"X 예시 : {X[0]}")
print(f"Y 예시 : {Y[0]}")

X 예시 : tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 

In [12]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [13]:
net = Net(dic_size, hidden_size, 2)
net

Net(
  (rnn): RNN(25, 25, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=25, out_features=25, bias=True)
)

In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr)

In [15]:
outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [16]:
print(outputs.view(-1, dic_size).shape)

torch.Size([1700, 25])


In [17]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [18]:
for i in range(150):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

gagao gopaa aagoaoygopfg gggpayogcyf oogofggaigfggpogoorg aafgayfyoaofagyafg  fwygoooopcotgooaogaougaoggfgggogafafoaoopaaogaatgoy fyokftygfgooayogofro gogoggoogaoaggttfgaaoo g goa
tfo tgtt grctrtttgg gt trgg tggrgagtct ttftggrgttct grgrrgtoggtggcggt trrrcttrtgttriag tggcggrrtttt rtgtgattrtrttt ragthrrrfrrtrcggrrgtrtggggrttg tt tatfgttgrttirtgo tat gttrgrtgc
tohth to  htt ttot  h hh thoto   h o t t  thh th th    ht thhht ttthhth ttthh  o tth th ohoth t tt  ho h ott tth tt hhhhet  h  t th t th to  htohtth ttt teoo  hh h  th th  h t ttt
nf' nfo  f'''ffff'.ffff'fff'f.f'..'ff''f'fff'ffyffmffffp'ffffffffofff.ffffff.'f'.ff.f.f''yf'.f'ff'.ffff.f'.ff'ff'fy.ff.fffff'fff'fof'fp'fff'fmf'.f'ff'fo'fff'ffff''fffff'ffffffgfk.
t to o o o  m mhsm o i heio o  a  o     o o         f  ei a i   m     o    f eae    t   t       a i  n   o odm  a.      e  ae o eai .o ee  'o   gtt .m   i  i ta ao  ii ai   et   i
t  a tu     o tt p   t   t n o t     t to t   to  a      ut  to tet  po   tn   o tu  t  o  o    uh  